<a href="https://colab.research.google.com/github/mostly-sunny/digital-health-hackathon/blob/main/coxph_find_best_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pycox - CoxPH Model
- Network : Test with sets, and find best network and lr by lowest brier score
- Input Variables : G1 ~ G300, Var1 ~ Var10, Treatment
- Output Variables : time, event
- Scaler : MinMaxScaler -> Var1 ~ Var10



In [96]:
pip install pycox

In [97]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn_pandas import DataFrameMapper
import pandas as pd

import torch
import torchtuples as tt

from pycox.models import CoxPH
from pycox.evaluation import EvalSurv

In [98]:
np.random.seed(123456)
_ = torch.manual_seed(123456)

- all-in-one.csv 파일은 유전자 변이 유무, 임상 변수, 생존 기간, 사망 여부, 치료 유무가 열로 존재하는 파일
- test-data-treat-and-untreat.csv 파일은 all-in-one의 열은 같은 602개의 데이터.
  - (0번째 행) : 유전자 변이 모두 0, 치료 0
  - (1번째 행) : 유전자 변이 모두 0, 치료 1
  - (2~301번째 행) : 유전자 변이 n-1에만 1, 치료 0
  - (302~601번재 행) : 유전자 변이 n-301에만 1, 치료 1
- pandas 라이브러리에 있는 csv 파일을 DataFrame으로 바꾸어주는 read_csv 함수를 이용하여 파일을 읽어 들임.
- DataFrame은 표를 나타내는 데이터 타입임.

In [99]:
dataset = pd.read_csv('/content/all-in-one.csv')
dataset_for_hr = pd.read_csv('/content/test-data-treat-and-untreat.csv')

- 위에서 읽어들인 dataset 중에서 20%는 검증(_val -> validation)을 위해 sampling 한다.
- 남은 80%의 데이터에서도 20%는 테스트(_test)를 위해 sampling 한다.

In [100]:
dataset_val = dataset.sample(frac=0.2)
dataset_train = dataset.drop(dataset_val.index)
dataset_test = dataset_train.sample(frac=0.2)
dataset_train = dataset_train.drop(dataset_test.index)

- columns_standardize : 임상변수 - 0~9사이의 값을 표준화
- columns_leave : 유전자 변이 유무 + 치료 유무 - 0과 1로 표현돼 있기 때문에 표준화 필요 없음.
- DataFrameMapper는 pandas DataFrame에서 원하는 열을 뽑아서 리스트로 만들어줌.
- 리스트로 만들때 StandardScaler()가 포함된 열은 표준화를 시킨 뒤, 그리고 None이면 갖고 있는 값을 그대로 넣음.

In [101]:
columns_standardize = ['Var' + str(i) for i in range(1,11)]
columns_leave = ['G' + str(i) for i in range(1,301)]
columns_leave += ['Treatment']

# standardize = [([col], StandardScaler()) for col in columns_standardize]
standardize = [([col], MinMaxScaler()) for col in columns_standardize]

leave = [(col, None) for col in columns_leave]

x_mapper = DataFrameMapper(leave + standardize)

- 위에서 만든 DataFrameMapper로 DataFrame 중 x(입력) 데이터를 모델이 학습할 수 있게끔 리스트 형식으로 바꾸어 준다.



In [102]:
x_train = x_mapper.fit_transform(dataset_train).astype('float32')
x_val = x_mapper.transform(dataset_val).astype('float32')
x_test = x_mapper.transform(dataset_test).astype('float32')
x_for_hr = x_mapper.transform(dataset_for_hr).astype('float32')

- DataFrame (표)에서 Y(출력)데이터인 time(생존기간)과 event(사망여부)를 뽑아 출력 데이터를 추린다.
- 검증(Validation)을 위한 입력-출력 세트 val을 만든다.

In [103]:
get_target = lambda df: (df['time'].values, df['event'].values)
y_train = get_target(dataset_train)
y_val = get_target(dataset_val)

durations_test, events_test = get_target(dataset_test)
val = x_val, y_val

함수 make_net : network을 생성해 리턴하는 함수
- input과 output의 노드 수, 은닉층 수, 은닉층의 노드 수 설정 가능

In [104]:
def make_net(in_features, out_features, hidden, nodes):
  if hidden == 1:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),
            
      torch.nn.Linear(nodes, out_features)
    )
  elif hidden == 2:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),
            
      torch.nn.Linear(nodes, out_features)
    )
  elif hidden == 3:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),
            
      torch.nn.Linear(nodes, out_features)
    )
  elif hidden == 4:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),
      
      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, out_features)
    )
  return network

- in_features : 입력데이터의 개수 (x_train.shape : 311 = 300(유전자) + 10(임상변수) + 1(치료유무))
- out_features : 출력노드의 개수

- hidden_layers : 은닉층 수를 가지고 있는 리스트
- number_nodes : 은닉층에 있는 노드 수를 가지고 있는 리스트
- learning_rates : 테스트할 학습률을 가지고 있는 리스트
- brier_scores = brier score을 계산해 append

In [105]:
in_features = x_train.shape[1]
out_features = 1

hidden_layers = [1,2,3,4]
number_nodes = [32, 64, 128, 256, 512, 1024, 2048]
learning_rates = [0.0001, 0.001, 0.01, 0.1]
brier_scores = []

for i in hidden_layers:
  for j in number_nodes:
    for k in learning_rates:
      net = make_net(in_features, out_features, i, j)
      model = CoxPH(net, tt.optim.Adam)

      batch_size = 256
      model.optimizer.set_lr(k)
      epochs = 512
      callbacks = [tt.callbacks.EarlyStopping()]
      verbose = True

      %%time
      model.fit(x_train, y_train, batch_size, epochs, callbacks, verbose, val_data=val, val_batch_size=batch_size)
      _ = model.compute_baseline_hazards()
      surv = model.predict_surv_df(x_test)
      
      # calculate ratio
      log_partial_hazard = model.predict(x_for_hr)
      partial_hazard = [np.exp(lph) for lph in log_partial_hazard]

      treat_hr = []
      # ratio with treated and untreated
      for g in range(300):
        treat_hr.append([partial_hazard[g+302]/partial_hazard[g+2],'G' + str(g+1)])
      treat_hr.sort()

      # evaluation
      ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')
      time_grid = np.linspace(durations_test.min(), durations_test.max(), 100)
      score = ev.integrated_brier_score(time_grid)
      brier_scores.append([score, i, j, k, treat_hr[:10]])

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.06 µs
0:	[0s / 0s],		train_loss: 4.4867,	val_loss: 4.2817
1:	[0s / 0s],		train_loss: 4.4710,	val_loss: 4.2825
2:	[0s / 0s],		train_loss: 4.4537,	val_loss: 4.2836
3:	[0s / 0s],		train_loss: 4.4543,	val_loss: 4.2850
4:	[0s / 0s],		train_loss: 4.4115,	val_loss: 4.2869
5:	[0s / 0s],		train_loss: 4.4026,	val_loss: 4.2893
6:	[0s / 0s],		train_loss: 4.3801,	val_loss: 4.2924
7:	[0s / 0s],		train_loss: 4.3459,	val_loss: 4.2963
8:	[0s / 0s],		train_loss: 4.3715,	val_loss: 4.3013
9:	[0s / 0s],		train_loss: 4.3589,	val_loss: 4.3072
10:	[0s / 0s],		train_loss: 4.3557,	val_loss: 4.3143
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.39 µs
0:	[0s / 0s],		train_loss: 4.5282,	val_loss: 4.2778
1:	[0s / 0s],		train_loss: 4.3406,	val_loss: 4.2756
2:	[0s / 0s],		train_loss: 4.2064,	val_loss: 4.2732
3:	[0s / 0s],		train_loss: 4.1309,	val_loss: 4.2700
4:	[0s / 0s],		train_loss: 4.0439,	val_loss: 4.2663
5:	[0s / 0s],		train_loss: 3.9997,	val_

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in true_divide


0:	[0s / 0s],		train_loss: 4.5158,	val_loss: 4.2946
1:	[0s / 0s],		train_loss: 4.3532,	val_loss: 4.2934
2:	[0s / 0s],		train_loss: 4.2185,	val_loss: 4.2917
3:	[0s / 0s],		train_loss: 4.1557,	val_loss: 4.2898
4:	[0s / 0s],		train_loss: 4.0478,	val_loss: 4.2876
5:	[0s / 0s],		train_loss: 4.0148,	val_loss: 4.2855
6:	[0s / 1s],		train_loss: 3.9378,	val_loss: 4.2838
7:	[0s / 1s],		train_loss: 3.8523,	val_loss: 4.2822
8:	[0s / 1s],		train_loss: 3.8222,	val_loss: 4.2813
9:	[0s / 1s],		train_loss: 3.7611,	val_loss: 4.2814
10:	[0s / 1s],		train_loss: 3.7026,	val_loss: 4.2839
11:	[0s / 1s],		train_loss: 3.6615,	val_loss: 4.2900
12:	[0s / 1s],		train_loss: 3.6312,	val_loss: 4.2986
13:	[0s / 1s],		train_loss: 3.6057,	val_loss: 4.3111
14:	[0s / 2s],		train_loss: 3.5321,	val_loss: 4.3271
15:	[0s / 2s],		train_loss: 3.4939,	val_loss: 4.3459
16:	[0s / 2s],		train_loss: 3.4620,	val_loss: 4.3669
17:	[0s / 2s],		train_loss: 3.4258,	val_loss: 4.3869
18:	[0s / 2s],		train_loss: 3.4236,	val_loss: 4.4079
CPU

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in true_divide



Wall time: 7.87 µs
0:	[0s / 0s],		train_loss: 4.4833,	val_loss: 4.2776
1:	[0s / 0s],		train_loss: 4.4764,	val_loss: 4.2778
2:	[0s / 0s],		train_loss: 4.4599,	val_loss: 4.2782
3:	[0s / 0s],		train_loss: 4.4555,	val_loss: 4.2788
4:	[0s / 0s],		train_loss: 4.4688,	val_loss: 4.2795
5:	[0s / 0s],		train_loss: 4.4279,	val_loss: 4.2809
6:	[0s / 0s],		train_loss: 4.4009,	val_loss: 4.2830
7:	[0s / 0s],		train_loss: 4.3920,	val_loss: 4.2861
8:	[0s / 0s],		train_loss: 4.3689,	val_loss: 4.2897
9:	[0s / 0s],		train_loss: 4.3649,	val_loss: 4.2944
10:	[0s / 0s],		train_loss: 4.3425,	val_loss: 4.2992
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs
0:	[0s / 0s],		train_loss: 4.4488,	val_loss: 4.2741
1:	[0s / 0s],		train_loss: 4.2958,	val_loss: 4.2721
2:	[0s / 0s],		train_loss: 4.2544,	val_loss: 4.2695
3:	[0s / 0s],		train_loss: 4.1390,	val_loss: 4.2665
4:	[0s / 0s],		train_loss: 4.0694,	val_loss: 4.2628
5:	[0s / 0s],		train_loss: 4.0486,	val_loss: 4.2581
6:	[0s / 0s],		train_loss: 3.973

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in true_divide


CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.91 µs
0:	[0s / 0s],		train_loss: 4.4159,	val_loss: 4.2730
1:	[0s / 1s],		train_loss: 3.9614,	val_loss: 4.2624
2:	[0s / 1s],		train_loss: 3.7450,	val_loss: 4.2497
3:	[0s / 2s],		train_loss: 3.5258,	val_loss: 4.2350
4:	[0s / 2s],		train_loss: 3.3450,	val_loss: 4.2184
5:	[0s / 3s],		train_loss: 3.2119,	val_loss: 4.1985
6:	[0s / 3s],		train_loss: 3.1302,	val_loss: 4.1765
7:	[0s / 4s],		train_loss: 3.0081,	val_loss: 4.1537
8:	[0s / 5s],		train_loss: 2.9128,	val_loss: 4.1333
9:	[0s / 5s],		train_loss: 2.8677,	val_loss: 4.1235
10:	[0s / 6s],		train_loss: 2.7824,	val_loss: 4.1362
11:	[0s / 6s],		train_loss: 2.7370,	val_loss: 4.1812
12:	[0s / 7s],		train_loss: 2.7261,	val_loss: 4.2443
13:	[0s / 7s],		train_loss: 2.6931,	val_loss: 4.3108
14:	[0s / 8s],		train_loss: 2.6651,	val_loss: 4.3955
15:	[0s / 8s],		train_loss: 2.6628,	val_loss: 4.4890
16:	[0s / 8s],		train_loss: 2.6831,	val_loss: 4.5662
17:	[0s / 9s],		train_loss: 2.6963,	val_loss:

- brier_score가 가장 작은 것부터 정렬


In [106]:
brier_scores.sort()
selected_genes = []
for i in range(10):
  selected_genes.append(brier_scores[0][4][i][1])

- brier_score가 가장 좋은 케이스 출력

In [107]:
print("Brier Score :", brier_scores[0][0])
print("Hidden Layers :", brier_scores[0][1])
print("Number of Nodes :", brier_scores[0][2])
print("Learning Rate :", brier_scores[0][3])
print("Selection :", selected_genes)

Brier Score : 0.055871205849768915
Hidden Layers : 4
Number of Nodes : 1024
Learning Rate : 0.001
Selection : ['G291', 'G263', 'G31', 'G119', 'G182', 'G222', 'G146', 'G212', 'G149', 'G172']


In [108]:
for i in brier_scores:
  print(i[:-1])

[0.055871205849768915, 4, 1024, 0.001]
[0.056523081907182526, 3, 512, 0.001]
[0.05705684692466478, 3, 256, 0.001]
[0.05709214137685976, 2, 2048, 0.001]
[0.05722871237036251, 2, 1024, 0.001]
[0.05747192219132218, 2, 512, 0.001]
[0.05760600449171411, 4, 512, 0.001]
[0.0581476399170425, 3, 1024, 0.001]
[0.05821185362939469, 3, 128, 0.01]
[0.05824877604535191, 2, 512, 0.01]
[0.058321449017705806, 3, 2048, 0.0001]
[0.05845158334839935, 1, 512, 0.001]
[0.05880587245309833, 1, 512, 0.01]
[0.05883102945184286, 2, 1024, 0.0001]
[0.0588775380801343, 2, 32, 0.001]
[0.05899472264937506, 2, 64, 0.01]
[0.05899698467054138, 1, 2048, 0.001]
[0.05904710068167665, 3, 256, 0.01]
[0.05916400253305097, 1, 1024, 0.001]
[0.05921495937584547, 1, 128, 0.01]
[0.059217291391708955, 2, 2048, 0.0001]
[0.059381161876021125, 2, 256, 0.001]
[0.05940087629354267, 2, 128, 0.01]
[0.05954602766683337, 4, 128, 0.01]
[0.05957080416299644, 2, 128, 0.001]
[0.05959761576385912, 1, 1024, 0.01]
[0.059608289669407004, 3, 1024, 0